# IMPORT DEPENDENCIES

In [2]:
# Add the parent directory of the current script to the system path
# to enable importing modules from that directory or its subdirectories.
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
# Import necessary modules and functions
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import pandas as pd
import geopandas as gpd
# Import custom modules and functions
from src.processing import read_outlets, join_watersheds2points, load_river_network, clip_river_network, insert_watershed_info, process_watershed_points
from src.snap_pour_point import read_flow_accumulation_tif, calculate_new_pour_point
from src.delineator import read_drainage_direction, calculate_upstream_v2  
from src.polygonize import raster_to_polygon, rasterize_array
from src.file_manager import create_results_directory

from configuration import OUTLETS, WATERSHEDS, MODE, FLOW_ACCUMULATION, DRAINAGE_DIRECTION, PIXEL2SEARCH, RIVERS, RESULTS, MAX_STRAHLER


In [3]:
create_results_directory(RESULTS)

points = read_outlets(OUTLETS)

if MODE == "single":

    accum, pixel_size = read_flow_accumulation_tif(FLOW_ACCUMULATION)

    drainage_direction, tif_profile, dr_dir_src = read_drainage_direction(DRAINAGE_DIRECTION)

    river_vector = load_river_network(RIVERS)   
    
    points_new = process_watershed_points(points, accum, pixel_size, drainage_direction, dr_dir_src,
                            tif_profile, river_vector, MAX_STRAHLER, RESULTS)
    
    points_new["change_rate[%]"] = 100 * (points_new["CalculatedArea[km2]"] - points_new["area[km2]"]) / points_new["area[km2]"]
    
elif MODE == "partial":

    points_labelled = join_watersheds2points(points, WATERSHEDS)
    unique_watershed_ids = points_labelled["Watershed_ID"].unique()
    points_new = pd.DataFrame()
    for watershed in unique_watershed_ids:
        print(f"{watershed}")
        filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed]    
        accum, pixel_size = read_flow_accumulation_tif(os.path.join(FLOW_ACCUMULATION, watershed + '.tif'))

        drainage_direction, tif_profile, dr_dir_src = read_drainage_direction(os.path.join(DRAINAGE_DIRECTION, watershed + '.tif'))

        river_vector = load_river_network(os.path.join(RIVERS, watershed + '.geojson'))   
        
        points_watershed = process_watershed_points(filtered_points_labelled, accum, pixel_size, drainage_direction, dr_dir_src,
                                tif_profile, river_vector, MAX_STRAHLER, RESULTS) 
        
        points_new = pd.concat([points_new, points_watershed], ignore_index=True)
        
    points_new["change_rate[%]"] = 100 * (points_new["CalculatedArea[km2]"] - points_new["area[km2]"]) / points_new["area[km2]"]
try:
    points_new.to_csv(os.path.join(RESULTS, 'report_secondRUN.csv'), encoding="windows-1254")
except UnicodeEncodeError:
    points_new.to_csv(os.path.join(RESULTS, 'report_secondRUN.csv'), encoding="utf8")

TR02
[+] Processing D02A031.
[+] Processing D02A123.
[+] Processing D02A173.
TR03
[+] Processing D03A049.
[+] Processing D03A115.


In [10]:
import fiona


In [1]:
from datetime import datetime

In [6]:
    
    # Get current date and time
    now = datetime.now()

    # Format to dd/mm/YYYY HH:SS
    formatted_datetime = now.strftime("%d%m%Y_%H%M")

In [8]:
datetime.now().strftime("%d%m%Y_%H%M")

'20092023_1554'

# DEBUGGING POLYGONIZE

In [ ]:


def calculate_upstream_v1(drainage_direction, pour_point_coords):
    """
    Calculate the upstream area based on the given drainage direction and pour point coordinates.

    Parameters:
    - drainage_direction: numpy.ndarray
        A 2D array representing the drainage direction. Each pixel indicates the direction in which water flows.
    - pour_point_coords: tuple (row, column)
        The row and column indices of the pour point, which is the starting point for calculating the upstream area.

    Returns:
    - upstream_area: numpy.ndarray
        A boolean array indicating the pixels that belong to the upstream area.

    The function uses a recursive algorithm to traverse upstream from the pour point, following the drainage direction.
    It marks the pixels that contribute flow to the pour point as part of the upstream area.

    Note:
    - The drainage direction values should follow the D8 convention (values 1 to 8 representing the eight directions).
    - The pour point coordinates should be within the boundaries of the drainage direction array.
    """

    # Create an empty array with the same shape as the drainage direction
    upstream_area = np.zeros_like(drainage_direction, dtype=bool)

    # Get the row and column indices of the pour point
    row, col = pour_point_coords

    # Define the D8 offsets for the eight directions
    offsets = [(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1)]

    # Recursive function to calculate upstream area
    def traverse_upstream(r, c):
        # Check if the current pixel is within the boundaries of the array
        if r < 0 or r >= drainage_direction.shape[0] or c < 0 or c >= drainage_direction.shape[1]:
            return

        # Check if the current pixel is already marked as part of the upstream area
        if upstream_area[r, c]:
            return

        # Mark the current pixel as part of the upstream area
        upstream_area[r, c] = True

        # Iterate over the eight directions
        for direction in range(8):
            # Get the offset for the corresponding direction
            offset = offsets[direction]

            # Move to the next pixel based on the offset
            next_r = r + offset[0]
            next_c = c + offset[1]

            # Check if the next pixel drains into the current pixel
            if next_r >= 0 and next_r < drainage_direction.shape[0] and next_c >= 0 and next_c < drainage_direction.shape[1]:
                if (drainage_direction[next_r, next_c] - 1) % 8 == (7 + direction - 4) % 8:
                    # Recursively traverse upstream from the next pixel
                    traverse_upstream(next_r, next_c)

    # Start traversing upstream from the pour point
    traverse_upstream(row, col)

    return upstream_area.astype(int)


In [ ]:
from numba import njit
@njit
def calculate_upstream_v2(drainage_direction, pour_point_coords):
    """
    Calculate the upstream area based on the given drainage direction and pour point coordinates.

    Parameters:
    - drainage_direction: numpy.ndarray
        A 2D array representing the drainage direction. Each pixel indicates the direction in which water flows.
    - pour_point_coords: tuple (row, column)
        The row and column indices of the pour point, which is the starting point fo r calculating the upstream area.

    Returns:
    - upstream_area: numpy.ndarray
        A binary array indicating the pixels that belong to the upstream area (1 for upstream, 0 for other pixels).

    The function uses a stack-based iterative algorithm to traverse upstream from the pour point, following the
    drainage direction. It marks the pixels that contribute flow to the pour point as part of the upstream area.

    Note:
    - The drainage direction values should follow the D8 convention (values 1 to 8 representing the eight directions).
    - The pour point coordinates should be within the boundaries of the drainage direction array.
    - The function utilizes the `numba` library's `njit` decorator for improved performance.
    """

    # Create an empty array with the same shape as the drainage direction
    upstream_area = np.zeros_like(drainage_direction, dtype=np.int8)

    # Get the row and column indices of the pour point
    row, col = pour_point_coords

    # Define the D8 offsets for the eight directions
    offsets = [(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1)]
    offsets = [(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1)]

    # Create a stack to store the pixel coordinates
    stack = [(row, col)]

    # Mark the pour point as part of the upstream area
    upstream_area[row, col] = 1

    # Iterate until the stack is empty
    while stack:
        r, c = stack.pop()
        print("r, c", r, c)

        # Iterate over the eight directions
        for direction in range(8):
            offset = offsets[direction]
            next_r = r + offset[0]
            next_c = c + offset[1]

            # Check if the next pixel is within the array boundaries
 
            if 0 <= next_r < drainage_direction.shape[0] and 0 <= next_c < drainage_direction.shape[1]:
                # if (drainage_direction[next_r, next_c] - 1) % 8 == (7 + direction - 4) % 8:
                if (drainage_direction[next_r, next_c] - 1) % 8 == (7 + direction - 4) % 8:
                    print(next_r, next_c)
                    upstream_area[next_r, next_c] = 1
                    stack.append((next_r, next_c))
        print("Stack:", len(stack))
    return upstream_area

In [ ]:
create_results_directory(RESULTS)

points = read_outlets(OUTLETS)
points_labelled = join_watersheds2points(points, WATERSHEDS)
unique_watershed_ids = points_labelled["Watershed_ID"].unique()
points = points_labelled.copy()


for index, row in points.iterrows():
    
    # if row.id == "D25A039":
    if row.id == "D25A045": 
    # if row.id == "D25A044": 
        print(f"[+] Processing {row.id}.")   
        # Calculate new pour point
        new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), 1)
        new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])

        # Extract watersheds
        upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
        rasterized_array = rasterize_array(upstream_area, tif_profile)

        # Save polygon and line as JSON
        subbasin = raster_to_polygon(rasterized_array, save_polygon=True, 
                                        polygon_save_path=os.path.join(RESULTS, "watershed", str(row.id) + "_basin"))

        # Clip rivers
        clipped_river_network, feedback = clip_river_network(river_vector, subbasin, 
                                                            max_strahler_order=MAX_STRAHLER, 
                                                            line_save_path=os.path.join(RESULTS, "river", str(row.id) + "_river"))

        # Insert watershed delineation information into the points table
        points_copy = insert_watershed_info(points_copy, row, new_pour_point, 
                                            subbasin["CalculatedArea[km2]"][0], feedback)

In [ ]:
subbasin.plot()

In [ ]:
points_copy

In [ ]:
_MODE = "partial"
if MODE == "partial":
    points_labelled = join_watersheds2points(points, WATERSHEDS)
    unique_watershed_ids = points_labelled["Watershed_ID"].unique()
    for watershed in unique_watershed_ids[:1]:
        filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 
    

In [ ]:
watershed

In [ ]:
for index, row in points.iterrows():
    print(f"[+] Proccessing {row.id}.")
    # Calculate new pour point
    new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
    new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
    # Extract watersheds
    upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
    rasterized_array = rasterize_array(upstream_area, tif_profile)
    # Save polygon and line as JSON
    subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
    # Clip rivers

    clipped_river_network, feedback = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))

    # Insert watershed delienation information into the points table
    points_copy = insert_watershed_info(points_copy, row, new_pour_point, subbasin["CalculatedArea[km2]"][0], feedback)


In [ ]:
process_watershed_points(points, accum, pixel_size, drainage_direction, dr_dir_src,
                            tif_profile, river_vector, MAX_STRAHLER, RESULTS)

In [ ]:
type(dr_dir_src)

In [ ]:
points

In [ ]:
def loop_over_points():
    for index, row in points.iterrows():
   
        # Calculate new pour point
        new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
        new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
        # Extract watersheds
        upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
        rasterized_array = rasterize_array(upstream_area, tif_profile)
        # Save polygon and line as JSON
        subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
        # Clip rivers
        clipped_river_network = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))


In [ ]:

for index, row in points.iterrows():
   
    # Calculate new pour point
    new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
    new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
    # Extract watersheds
    upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
    rasterized_array = rasterize_array(upstream_area, tif_profile)
    # Save polygon and line as JSON
    subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
    # Clip rivers
    clipped_river_network = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))
      

In [ ]:
def read_drainage_direction(drainage_direction_path, outlets=None, pour_point_coord=None):
    """
    Reads the drainage direction data from a TIFF file.

    Parameters:
        drainage_direction_path (str): File path of the drainage direction TIFF.
        outlets (geopandas.GeoDataFrame, optional): GeoDataFrame containing outlets with 'long' and 'lat' columns.
            Each outlet's coordinates will be used as pour points to locate specific cells in the TIFF.
        pour_point_coord (tuple, optional): Coordinates of the pour point in the format (x, y).
            The pour point coordinates are used to locate the specific cell in the TIFF.

    Returns:
        tuple: A tuple containing the drainage direction data as a NumPy array,
               the pour point coordinates in the TIFF, and the metadata profile of the TIFF.

    Notes:
        - The function uses the rasterio library to read the TIFF file.
        - The drainage direction data is typically represented as an array where each
          cell represents the direction of flow.
        - If `pour_point_coord` is provided, it is used as the pour point to locate the specific cell in the TIFF.
        - If `outlets` is provided, each outlet's coordinates are used as pour points to locate specific cells in the TIFF.
        - The metadata profile contains information about the TIFF such as its spatial
          reference system, resolution, and other properties.

    Raises:
        ValueError: If both `outlets` and `pour_point_coord` are provided at the same time, or if `outlets` is not a GeoDataFrame.
    """
    if outlets is not None and pour_point_coord is not None:
        raise ValueError("Only one of 'outlets' or 'pour_point_coord' should be provided, not both.")
    if outlets is not None and (outlets.empty or outlets is None):
        raise ValueError("'outlets' GeoDataFrame is empty or None.")
    if outlets is not None and not isinstance(outlets, gpd.GeoDataFrame):
        raise ValueError("'outlets' must be a GeoDataFrame.")

    with rasterio.open(drainage_direction_path) as src:
        drainage_direction = src.read(1)
        profile = src.profile
        if pour_point_coord:
            pour_point_xy = src.index(pour_point_coord[0], pour_point_coord[1])
            return drainage_direction, pour_point_xy, profile
        if outlets is not None:
            for index, row in outlets.iterrows():
                pour_point_coord = row.long, row.lat
                pour_point_xy = src.index(pour_point_coord[0], pour_point_coord[1])
                outlets.loc[index, 'pour_point_x'] = int(pour_point_xy[0])
                outlets.loc[index, 'pour_point_y'] = int(pour_point_xy[1])
            return drainage_direction, outlets, profile

# SET THE PARAMETERS

In [ ]:
# Number of neighboring pixels to consider 
# in search of the neigboring pixel with the highest flow accumualtion
PIXEL2SEARCH = 1

# CALCULATION

In [ ]:
path = '../data/flow_accumulation_TR.tif'
coord = (28.864, 40.130) 
coord = (28.963, 40.118) 

data, pixel_size = read_flow_accumulation_tif(path)
new_pour_point = calculate_new_pour_point(data, pixel_size, coord, PIXEL2SEARCH)
print(new_pour_point)

In [ ]:
drainage_direction_path = "../data/drainage_direction_TR.tif"
dr_dir, tif_profile, dr_dir_src = read_drainage_direction(drainage_direction_path)
pour_point_xy  = dr_dir_src.index(new_pour_point[0, new_pour_point[1]])
# Calculate upstream area
upstream_area = calculate_upstream_v2(dr_dir, pour_point_xy)

rasterized_array = rasterize_array(upstream_area, tif_profile, )
subbasin = raster_to_polygon(rasterized_array, save_polygon=False, polygon_save_path="output/polygon")

In [ ]:
subbasin.plot()

In [ ]:
watershed_id = "TR03"

In [ ]:
river_vector = load_river_network(watershed_id) 

In [ ]:
clipped_river_network = clip_river_network(river_vector, subbasin, 'output/river.geojson')

# TEST

## Read points

In [ ]:
points = read_outlets(OUTLETS)

In [ ]:
points.plot()

In [ ]:
points_labelled = join_watersheds2points(points, WATERSHEDS)
unique_watershed_ids = points_labelled["Watershed_ID"].unique()
for watershed in unique_watershed_ids[:1]:
    filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 

In [ ]:
for watershed in unique_watershed_ids[:1]:
    filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 

In [ ]:
filtered_points_labelled

In [ ]:
for index, row in filtered_points_labelled.iterrows():
    coord = [row["long"], row["lat"]]
    
    
    

In [ ]:
points = read_outlets(OUTLETS)

# if MODE == "single":
    